### Regularization Assignments

Use the model below as a basis for comparison to your regularized models.

In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from sklearn.metrics import r2_score as r2
from sklearn.metrics import mean_absolute_error as mae


computers = pd.read_csv("../Data/Computers.csv")

computers.tail()

,price,speed,hd,ram,screen,cd,multi,premium,ads,trend
6254,1690,100,528,8,15,no,no,yes,39,35
6255,2223,66,850,16,15,yes,yes,yes,39,35
6256,2654,100,1200,24,15,yes,no,yes,39,35
6257,2195,100,850,16,15,yes,no,yes,39,35
6258,2490,100,850,16,17,yes,no,yes,39,35


In [2]:
computers_eng = (
    pd.get_dummies(
        computers.assign(
            hd2 = computers["hd"] ** 2,
            hd3 = computers["hd"] ** 3,
        ),
    drop_first=True    
    )
)

In [3]:
from sklearn.model_selection import train_test_split

X = sm.add_constant(computers_eng.drop("price", axis=1))
y = np.log(computers["price"])

# Test Split

X, X_test, y, y_test = train_test_split(X, y, test_size=.2, random_state=12345)

In [5]:
model = sm.OLS(y, X).fit()

#model.summary()

print(f"Training R2: {r2(y, model.predict(X))}")
print(f"Training MAE: {mae(y, model.predict(X))}")

Training R2: 0.8049349026307837
Training MAE: 0.0873750253656206


In [6]:
print(f"Test R2: {r2(y_test, model.predict(X_test))}")
print(f"Test MAE: {mae(y_test, model.predict(X_test))}")

Test R2: 0.8106348481860446
Test MAE: 0.08744222739943998


## Assignment 1: Ridge Regression

Fit a ridge regression model using ridgeCV and compare the accuracy to the model above. 

Don't forget to standardize your data!

In [7]:
# standardization

In [8]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
X_tr = std.fit_transform(X.values)
X_te = std.transform(X_test.values)

In [9]:
from sklearn.linear_model import RidgeCV

n_alphas = 200
alphas = 10 ** np.linspace(-3, 3, n_alphas)

ridge_model = RidgeCV(alphas=alphas, cv=5)

ridge_model.fit(X_tr, y)
print(ridge_model.score(X_tr, y))
print(mae(y, ridge_model.predict(X_tr)))
print(ridge_model.alpha_)

0.8048079003150022
0.08735780764699269
2.2219468609395236


In [10]:
print(ridge_model.score(X_te, y_test))
print(mae(y_test, ridge_model.predict(X_te)))

0.8114548776779424
0.0872495791435326


# Assignment 2: Lasso Regression

Fit a lasso regression model using LassoCV and compare the accuracy to the models above! 

Did any coefficient values drop to zero?

In [11]:
from sklearn.linear_model import LassoCV

n_alphas = 200
alphas = 10 ** np.linspace(-3, 3, n_alphas)

lasso_model = LassoCV(alphas=alphas, cv=5)

lasso_model.fit(X_tr, y)
print(lasso_model.score(X_tr, y))
print(mae(y, lasso_model.predict(X_tr)))
print(lasso_model.alpha_)

0.7990178822046732
0.08856024399732959
0.001


In [12]:
print(lasso_model.score(X_te, y_test))

0.8035706909339283


In [13]:
list(zip(X.columns, lasso_model.coef_))

[('const', 0.0),
 ('speed', 0.08521336907201489),
 ('hd', 0.1804037620442454),
 ('ram', 0.11281918306801339),
 ('screen', 0.045432558598861916),
 ('ads', 0.014417710738461107),
 ('trend', -0.18697834088414725),
 ('hd2', -0.09013210308200778),
 ('hd3', 0.0),
 ('cd_yes', 0.019337837392440913),
 ('multi_yes', 0.01610256509024398),
 ('premium_yes', -0.06925629782173345)]

## Assignment 3: Elastic Net Regression

Fit an elastic net regression model using ElasticNetCV and compare the accuracy to the models above! 

What was the optimal L1 Ratio?

In [14]:
from sklearn.linear_model import ElasticNetCV

alphas = 10 ** np.linspace(-3, 3, 200)
l1_ratios = np.linspace(.01, 1, 10)

enet_model = ElasticNetCV(alphas=alphas, l1_ratio=l1_ratios, cv=5)

enet_model.fit(X_tr, y)
print(enet_model.score(X_tr, y))
print(enet_model.score(X_te, y_test))
print(enet_model.alpha_)
print(enet_model.l1_ratio_)

0.8043966703829291
0.8116153465368512
0.001
0.01
